In [20]:
import json
import urllib.request
from urllib.error import HTTPError
from tqdm.auto import tqdm
import time
import collections

In [2]:
def myurlopen(url, max_wait=300):
    i = 2
    while True:
        try:
            req = urllib.request.Request(
                                            url,
                                            data=None,
                                            headers={
                                                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:108.0) Gecko/20100101 Firefox/108.0'
                                            }
                                        )
            r = urllib.request.urlopen(req)
            return r
        except HTTPError as e:
            print(e)
            sleep_time = 2 + 2**i / 1000
            if sleep_time > max_wait:
                print('ABORT!')
                raise TimeoutError(f'Timed out with max wait time of {max_wait} s')
            print(f'Some HTTP error occurred. Waiting for {sleep_time}...')
            time.sleep(sleep_time)
            i += 1
            continue

In [86]:
franchise = 'smite'
franchise_name = 'Smite'

franchises_file = './Franchises.yaml'
franchises_text = """
'<[model][illu]>source_anime, __Characters/${FRANCHISE}__**(yellow skin, orange skin, red skin:1.5)**':
  Description:
    - ${FRANCHISE} Characters
  Tags:
    - Characters
    - ${FRANCHISE}
"""
characterFile = f'./characters/{franchise_name}.txt'

In [ ]:
url = f"https://api.r34.app/booru/rule34.xxx/posts?baseEndpoint=rule34.xxx&limit=100&tags={franchise}&pageID=0"
response = myurlopen(url)
rjson = json.loads(response.read())

jdata = []
while(rjson['data']):
    # scrape the post data here
    jdata += rjson['data']
    url = rjson['links']['next']
    response = myurlopen(url)
    rjson = json.loads(response.read())
    time.sleep(1)
    print(f'now working on url: {url}')

now working on url: https://api.r34.app/booru/rule34.xxx/posts?baseEndpoint=rule34.xxx&limit=100&tags=smite&pageID=1
now working on url: https://api.r34.app/booru/rule34.xxx/posts?baseEndpoint=rule34.xxx&limit=100&tags=smite&pageID=2
now working on url: https://api.r34.app/booru/rule34.xxx/posts?baseEndpoint=rule34.xxx&limit=100&tags=smite&pageID=3
now working on url: https://api.r34.app/booru/rule34.xxx/posts?baseEndpoint=rule34.xxx&limit=100&tags=smite&pageID=4
now working on url: https://api.r34.app/booru/rule34.xxx/posts?baseEndpoint=rule34.xxx&limit=100&tags=smite&pageID=5
now working on url: https://api.r34.app/booru/rule34.xxx/posts?baseEndpoint=rule34.xxx&limit=100&tags=smite&pageID=6
now working on url: https://api.r34.app/booru/rule34.xxx/posts?baseEndpoint=rule34.xxx&limit=100&tags=smite&pageID=7
now working on url: https://api.r34.app/booru/rule34.xxx/posts?baseEndpoint=rule34.xxx&limit=100&tags=smite&pageID=8
now working on url: https://api.r34.app/booru/rule34.xxx/posts?b

In [27]:
character_tag_dict = {}

for post in jdata:
    post_tags = post['tags']
    characters = post_tags['character']

    for c in characters:
        if c in character_tag_dict.keys():
            character_tag_dict[c]['tags'] += post_tags['general']
            character_tag_dict[c]['franchise'] += post_tags['copyright']
            character_tag_dict[c]['post_count'] += 1
        else:
            character_tag_dict[c] = {
                'tags': post_tags['general'],
                'franchise': post_tags['copyright'],
                'post_count': 1
            }

In [40]:
related_tags_dict = {}

for character in character_tag_dict.keys():
   total_posts = character_tag_dict[character]['post_count']
   tag_counts = collections.Counter(character_tag_dict[character]['tags'])
   tag_frequency = {key: tag_counts[key] / total_posts for key in tag_counts.keys()}
   franchise_counts = collections.Counter(character_tag_dict[character]['franchise'])
   franchise_frequency = {key: franchise_counts[key] / total_posts for key in franchise_counts.keys()}
   related_tags_dict[character] = {
      'total_posts': total_posts,
      'counts': tag_counts,
      'frequencies': tag_frequency,
      'franchise_coun': franchise_counts,
      'franchise_freq': franchise_frequency
   }

In [57]:
accept_criteria = [
    'female',
    '1girl',
    '1girls'
]

reject_criteria = [
    'male',
    '1boy',
    '1boys'
]

In [ ]:
accepted = []
reject_if_below_threshold = 0.7
reject_if_above_threshold = 0.4
franchise_overlap_threshold = 0.9
total_post_threshold = 10
tag_overlap = 0.66

for c in related_tags_dict.keys():
    if related_tags_dict[c]['franchise_freq']['smite'] < franchise_overlap_threshold:
        print(f'Rejected {c}')
        continue
    if related_tags_dict[c]['total_posts'] < total_post_threshold:
        print(f'Rejected {c}')
        print(f'Post_count {related_tags_dict[c]['total_posts']}')
        continue
    hits = 0
    for a_tag in accept_criteria:
        if a_tag in related_tags_dict[c]['frequencies'].keys():
            if related_tags_dict[c]['frequencies'][a_tag] < reject_if_below_threshold:
                hits += 1
                print(f'tag {a_tag} below {reject_if_below_threshold} ({related_tags_dict[c]['frequencies'][a_tag]})')
    for r_tag in reject_criteria:
        if r_tag in related_tags_dict[c]['frequencies'].keys():
            if related_tags_dict[c]['frequencies'][r_tag] > reject_if_above_threshold:
                hits += 1
                print(f'tag {r_tag} above {reject_if_above_threshold} ({related_tags_dict[c]['frequencies'][r_tag]})')
    if hits == 0:
        accepted.append(c)
        print(f'Accepted {c}')
    else:
        print(f'Rejected {c}')

tag female below 0.7 (0.4020618556701031)
tag 1girls below 0.7 (0.422680412371134)
Rejected fenrir_(smite)
tag 1boy above 1.7 (3.3333333333333335)
Rejected cu_chulainn_(smite)
tag 1boy above 1.7 (2.6315789473684212)
Rejected loki_(smite)
Accepted the_morrigan_(smite)
Rejected hecate_(smite)
Post_count 9
Accepted neith_(smite)
Accepted nemesis_(smite)
Rejected nemesis_(resident_evil)
Post_count 1
tag 1girl below 0.7 (0.018292682926829267)
Rejected artemis_(smite)
Accepted classic_artemis
Rejected smite_(character)
Post_count 2
Accepted bellona_(smite)
tag 1girl below 0.7 (0.007042253521126761)
Rejected izanami_(smite)
Accepted nox_(smite)
Accepted artio_(smite)
Accepted aphrodite_(smite)
Accepted morgan_le_fay_(smite)
Rejected nut_(smite)
Post_count 8
Accepted nu_wa_(smite)
tag 1girl below 0.7 (0.14285714285714285)
Rejected artemis
Accepted kali_(smite)
tag 1girl below 0.7 (0.009523809523809525)
Rejected sol_(smite)
Accepted yemoja_(smite)
Accepted athena_(smite)
Accepted hel_(smite)
ta

In [84]:
characters = []

for c in accepted:
    added_tags = []
    for tag in related_tags_dict[c]['frequencies'].keys():
        if related_tags_dict[c]['frequencies'][tag] < tag_overlap:
            continue
        if tag.endswith('eyes') or tag.endswith('hair') or ('skin' in tag) or ('tail' in tag) or tag.endswith('ears') or ('ahoge' in tag):
            added_tags.append(tag)
    added_tags = set(added_tags)
    processed = [ x.strip().replace('_', ' ').replace('(', '\\(').replace(')', '\\)') for x in added_tags]
    upcycled_tags = ', '.join(processed)

    character = c.strip().replace('_', ' ').replace('(', '\\(').replace(')', '\\)')
    the_franchise = franchise.strip().replace('_', ' ').replace('(', '\\(').replace(')', '\\)')
    final_string = f'({character}:1.3), {the_franchise}, {upcycled_tags}\n'
    characters.append(final_string)

In [87]:
if len(characters) > 0:
    with open(characterFile, 'a+') as f:
        f.writelines( characters )
    with open(franchises_file, 'a+') as f:
        new_text = franchises_text.replace('${FRANCHISE}', franchise_name)
        f.write(new_text)